# Education Management System

In [2]:
import sqlite3

def create_database():
    conn = sqlite3.connect('education_management.db')
    cursor = conn.cursor()

    # Create Students table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Students (
        sid CHAR(8) PRIMARY KEY NOT NULL,
        name TEXT NOT NULL,
        age INTEGER NOT NULL CHECK(age >= 0 AND age <= 120),
        gender TEXT NOT NULL,
        country_code TEXT NOT NULL,
        contact_no TEXT NOT NULL,
        email TEXT NOT NULL
    )
    ''')

    # Create Teachers table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Teachers (
        id CHAR(8) PRIMARY KEY NOT NULL,
        name TEXT NOT NULL,
        age INTEGER NOT NULL CHECK(age >= 0 AND age <= 120),
        gender TEXT NOT NULL,
        country_code TEXT NOT NULL,
        contact_no TEXT NOT NULL,
        email TEXT NOT NULL
    )
    ''')

    # Create Courses table with NULLability for teacher_id
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Courses (
        course_code CHAR(9) PRIMARY KEY NOT NULL,
        course_name TEXT NOT NULL,
        teacher_id CHAR(8) DEFAULT NULL,
        FOREIGN KEY (teacher_id) REFERENCES Teachers (id) ON DELETE SET NULL
    )
    ''')

    # Create Enrolled table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Enrolled (
        course_code CHAR(9) NOT NULL,
        sid CHAR(8) NOT NULL,
        PRIMARY KEY (course_code, sid),
        FOREIGN KEY (course_code) REFERENCES Courses (course_code) ON DELETE SET NULL,
        FOREIGN KEY (sid) REFERENCES Students (sid) ON DELETE SET NULL
    )
    ''')

    conn.commit()
    conn.close()

# Call the function to create tables without dropping them
create_database()

In [3]:
import re
import tkinter as tk
from tkinter import ttk, messagebox

# Function to open 'Manage Students' window
def manage_students_window():
    students_window = tk.Toplevel(root)
    students_window.title("Manage Students")

    labels = ['SID', 'Name', 'Age', 'Gender', 'Country Code', 'Contact No', 'Email']
    entries = {}

    for i, label_text in enumerate(labels):
        label = tk.Label(students_window, text=label_text)
        label.grid(row=i, column=0, padx=10, pady=5, sticky='e')
        entry = tk.Entry(students_window)
        
        # Add validation for Contact No to allow only integers
        if label_text == 'Contact No':
            validate_cmd = students_window.register(validate_integer_input)
            entry.config(validate='key', validatecommand=(validate_cmd, '%P'))
        
        entry.grid(row=i, column=1, padx=10, pady=5)
        entries[label_text] = entry
    
    # Create dropdown for gender
    gender_label = tk.Label(students_window, text="Gender")
    gender_label.grid(row=3, column=0, padx=10, pady=5, sticky='e')
    gender_combobox = ttk.Combobox(students_window, values=["Male", "Female", "Other", "Not Applicable"])
    gender_combobox.grid(row=3, column=1, padx=10, pady=5)
    entries['Gender'] = gender_combobox

    # Buttons for Insert, View, Delete, and Update
    insert_button = tk.Button(students_window, text="Insert", command=lambda: insert_student(entries))
    insert_button.grid(row=len(labels), column=0, pady=20)

    view_button = tk.Button(students_window, text="View", command=view_students)
    view_button.grid(row=len(labels), column=1, pady=20)

    delete_button = tk.Button(students_window, text="Delete", command=lambda: delete_student(entries['SID'].get()))
    delete_button.grid(row=len(labels)+1, column=0, pady=20)

    update_button = tk.Button(students_window, text="Update", command=lambda: update_student(entries))
    update_button.grid(row=len(labels)+1, column=1, pady=20)

# Function to validate integer input
def validate_integer_input(P):
    if P == "" or P.isdigit():
        return True
    else:
        return False
        
# Function for email validation
def is_valid_email(email):
    return re.match(r"[^@]+@[^@]+\.[^@]+", email) is not None

# Function to insert student records with validation
def insert_student(entries):
    sid = entries['SID'].get().strip()
    
    # Validate SID length
    if len(sid) != 8:
        messagebox.showerror('Error', 'SID must be exactly 8 characters long!')
        return

    name = entries['Name'].get().strip()
    
    age_str = entries['Age'].get().strip()
    
    gender = entries['Gender'].get().strip()
    
    country_code = entries['Country Code'].get().strip()
    
    contact_no = entries['Contact No'].get().strip()
    
    email = entries['Email'].get().strip()

    # Check if all fields are filled
    if not all([sid, name, age_str, gender, country_code, contact_no, email]):
        messagebox.showerror('Error', 'Please fill in all fields!')
        return

    # Validate email format
    if not is_valid_email(email):
        messagebox.showerror('Error', 'Invalid email format!')
        return

    # Validate age limit (must be between 0 and 120 inclusive)
    if not age_str.isdigit() or not (0 <= int(age_str) <= 120):
        messagebox.showerror('Error', 'Age must be a number between 0 and 120!')
        return

    # Insert into database (actual insertion logic)
    try:
        conn = sqlite3.connect('education_management.db')
        cursor = conn.cursor()
        
        cursor.execute('''
            INSERT INTO Students (sid, name, age, gender, country_code, contact_no, email)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (sid, name, int(age_str), gender, country_code, contact_no, email))

        conn.commit()
        messagebox.showinfo('Success', f'Successfully inserted {name} into the database.')
        
    except sqlite3.IntegrityError:
        messagebox.showerror('Error', 'SID must be unique!')
    
    except Exception as e:
        messagebox.showerror('Error', str(e))
    
    finally:
        conn.close()

# Function to delete a student record by SID
def delete_student(sid):
    if not sid:
        messagebox.showerror('Error', 'Please enter a SID to delete!')
        return

    try:
        conn = sqlite3.connect('education_management.db')
        cursor = conn.cursor()

        cursor.execute('''
            DELETE FROM Students WHERE sid = ?
        ''', (sid,))
        
        if cursor.rowcount == 0:
            messagebox.showerror('Error', 'SID not found!')
        else:
            messagebox.showinfo('Success', f'Successfully deleted student with SID {sid}.')

        conn.commit()
        
    except Exception as e:
        messagebox.showerror('Error', str(e))
    
    finally:
        conn.close()

# Function to update student records
def update_student(entries):
    sid = entries['SID'].get().strip()
    
    if not sid:
        messagebox.showerror('Error', 'Please enter a valid SID to update!')
        return

    try:
        conn = sqlite3.connect('education_management.db')
        cursor = conn.cursor()

        # Check if SID exists in the database
        cursor.execute("SELECT * FROM Students WHERE sid = ?", (sid,))
        existing_record = cursor.fetchone()
        
        if not existing_record:
            messagebox.showerror('Error', f'Student with SID {sid} does not exist!')
            return

        # Retrieve new values from entries (use original values if blank)
        name = entries['Name'].get().strip() or existing_record[1]
        age_str = entries['Age'].get().strip()
        age = int(age_str) if age_str.isdigit() else existing_record[2]
        gender = entries['Gender'].get().strip() or existing_record[3]
        country_code = entries['Country Code'].get().strip() or existing_record[4]
        contact_no = entries['Contact No'].get().strip() or existing_record[5]
        email = entries['Email'].get().strip() or existing_record[6]

        # Validate email format if updated
        if email != existing_record[6] and not is_valid_email(email):
            messagebox.showerror('Error', 'Invalid email format!')
            return

        # Validate age limit (must be between 0 and 120 inclusive)
        if not (0 <= age <= 120):
            messagebox.showerror('Error', 'Age must be a number between 0 and 120!')
            return

        # Update record in the database
        cursor.execute('''
            UPDATE Students
            SET name = ?, age = ?, gender = ?, country_code = ?, contact_no = ?, email = ?
            WHERE sid = ?
            ''', (name, age, gender, country_code, contact_no, email, sid))

        conn.commit()
        messagebox.showinfo('Success', f'Successfully updated student with SID {sid}.')
    
    except Exception as e:
        messagebox.showerror('Error', str(e))
    
    finally:
        conn.close()

# Function to view student records
def view_students():
    conn = sqlite3.connect('education_management.db')
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM Students")
    rows = cursor.fetchall()

    # Create a new window to display the records
    view_window = tk.Toplevel(root)
    view_window.title("Students Table")

    # Create a treeview to display the data
    tree = ttk.Treeview(view_window)
    
    # Define columns
    tree["columns"] = ("SID", "Name", "Age", "Gender", "Country Code", "Contact No", "Email")
    
    # Format columns and set widths
    column_widths = {
        'SID': 100,
        'Name': 150,
        'Age': 50,
        'Gender': 100,
        'Country Code': 100,
        'Contact No': 100,
        'Email': 200
    }

    for col in tree["columns"]:
        tree.heading(col, text=col)
        tree.column(col, anchor="center", width=column_widths[col])  # Set width for each column

    # Insert data into treeview
    for row in rows:
        tree.insert("", "end", values=row)

    tree.pack(expand=True, fill='both')

In [4]:
# Function to open 'Manage Teachers' window
def manage_teachers_window():
    teachers_window = tk.Toplevel(root)
    teachers_window.title("Manage Teachers")

    labels = ['ID', 'Name', 'Age', 'Gender', 'Country Code', 'Contact No', 'Email']
    entries = {}

    for i, label_text in enumerate(labels):
        label = tk.Label(teachers_window, text=label_text)
        label.grid(row=i, column=0, padx=10, pady=5, sticky='e')
        entry = tk.Entry(teachers_window)
        
        # Add validation for Contact No to allow only integers
        if label_text == 'Contact No':
            validate_cmd = teachers_window.register(validate_integer_input)
            entry.config(validate='key', validatecommand=(validate_cmd, '%P'))
        
        entry.grid(row=i, column=1, padx=10, pady=5)
        entries[label_text] = entry

    # Create dropdown for gender
    gender_label = tk.Label(teachers_window, text="Gender")
    gender_label.grid(row=3, column=0, padx=10, pady=5, sticky='e')
    gender_combobox = ttk.Combobox(teachers_window, values=["Male", "Female", "Other", "Not Applicable"])
    gender_combobox.grid(row=3, column=1, padx=10, pady=5)
    entries['Gender'] = gender_combobox

    # Buttons for Insert, View, Delete, and Update
    insert_button = tk.Button(teachers_window, text="Insert", command=lambda: insert_teacher(entries))
    insert_button.grid(row=len(labels), column=0, pady=20)

    view_button = tk.Button(teachers_window, text="View", command=view_teachers)
    view_button.grid(row=len(labels), column=1, pady=20)

    delete_button = tk.Button(teachers_window, text="Delete", command=lambda: delete_teacher(entries['ID'].get()))
    delete_button.grid(row=len(labels)+1, column=0, pady=20)

    update_button = tk.Button(teachers_window, text="Update", command=lambda: update_teacher(entries))
    update_button.grid(row=len(labels)+1, column=1, pady=20)

# Function to validate integer input
def validate_integer_input(P):
    return P == "" or P.isdigit()
    
# Function for email validation
def is_valid_email(email):
    return re.match(r"[^@]+@[^@]+\.[^@]+", email) is not None

# Function to insert teacher records with validation
def insert_teacher(entries):
    id = entries['ID'].get().strip()
    
    # Validate ID length
    if len(id) != 8:
        messagebox.showerror('Error', 'ID must be exactly 8 characters long!')
        return

    name = entries['Name'].get().strip()
    
    age_str = entries['Age'].get().strip()
    
    gender = entries['Gender'].get().strip()
    
    country_code = entries['Country Code'].get().strip()
    
    contact_no = entries['Contact No'].get().strip()
    
    email = entries['Email'].get().strip()

    # Check if all fields are filled
    if not all([id, name, age_str, gender, country_code, contact_no, email]):
        messagebox.showerror('Error', 'Please fill in all fields!')
        return

    # Validate email format
    if not is_valid_email(email):
        messagebox.showerror('Error', 'Invalid email format!')
        return

    # Validate age limit (must be between 0 and 120 inclusive)
    if not age_str.isdigit() or not (0 <= int(age_str) <= 120):
        messagebox.showerror('Error', 'Age must be a number between 0 and 120!')
        return

    # Insert into database (actual insertion logic)
    try:
        conn = sqlite3.connect('education_management.db')
        cursor = conn.cursor()
        
        cursor.execute('''
            INSERT INTO Teachers (id, name, age, gender, country_code, contact_no, email)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (id, name, int(age_str), gender, country_code, contact_no, email))

        conn.commit()
        messagebox.showinfo('Success', f'Successfully inserted {name} into the database.')
        
    except sqlite3.IntegrityError:
        messagebox.showerror('Error', 'ID must be unique!')
    
    except Exception as e:
        messagebox.showerror('Error', str(e))
    
    finally:
        conn.close()

# Function to delete a teacher record by ID
def delete_teacher(teacher_id):
    if not teacher_id:
        messagebox.showerror('Error', 'Please enter a Teacher ID to delete!')
        return

    try:
        conn = sqlite3.connect('education_management.db')
        cursor = conn.cursor()

        cursor.execute('DELETE FROM Teachers WHERE id = ?', (teacher_id,))
        
        if cursor.rowcount == 0:
            messagebox.showerror('Error', 'Teacher ID not found!')
        else:
            messagebox.showinfo('Success', f'Successfully deleted teacher with ID {teacher_id}.')

        conn.commit()
    
    except Exception as e:
        messagebox.showerror('Error', str(e))
    
    finally:
        conn.close()

# Function to update teacher records
def update_teacher(entries):
    id = entries['ID'].get().strip()
    
    if not id:
        messagebox.showerror('Error', 'Please enter a valid Teacher ID to update!')
        return

    try:
        conn = sqlite3.connect('education_management.db')
        cursor = conn.cursor()

        # Check if ID exists in the database
        cursor.execute("SELECT * FROM Teachers WHERE id = ?", (id,))
        existing_record = cursor.fetchone()
        
        if not existing_record:
            messagebox.showerror('Error', f'Teacher with ID {id} does not exist!')
            return

        # Retrieve new values from entries (use original values if blank)
        name = entries['Name'].get().strip() or existing_record[1]
        age_str = entries['Age'].get().strip()
        age = int(age_str) if age_str.isdigit() else existing_record[2]
        gender = entries['Gender'].get().strip() or existing_record[3]
        country_code = entries['Country Code'].get().strip() or existing_record[4]
        contact_no = entries['Contact No'].get().strip() or existing_record[5]
        email = entries['Email'].get().strip() or existing_record[6]

        # Validate email format if updated
        if email != existing_record[6] and not is_valid_email(email):
            messagebox.showerror('Error', 'Invalid email format!')
            return

        # Validate age limit (must be between 0 and 120 inclusive)
        if not (0 <= age <= 120):
            messagebox.showerror('Error', 'Age must be a number between 0 and 120!')
            return

        # Update record in the database
        cursor.execute('''
            UPDATE Teachers
            SET name = ?, age = ?, gender = ?, country_code = ?, contact_no = ?, email = ?
            WHERE id = ?
            ''', (name, age, gender, country_code, contact_no, email, id))

        conn.commit()
        messagebox.showinfo('Success', f'Successfully updated teacher with ID {id}.')
    
    except Exception as e:
        messagebox.showerror('Error', str(e))
    
    finally:
        conn.close()

# Function to view teacher records
def view_teachers():
    conn = sqlite3.connect('education_management.db')
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM Teachers")
    rows = cursor.fetchall()

    # Create a new window to display the records
    view_window = tk.Toplevel(root)
    view_window.title("Teachers Table")

    # Create a treeview to display the data
    tree = ttk.Treeview(view_window)
    
    # Define columns
    tree["columns"] = ("ID", "Name", "Age", "Gender", "Country Code", "Contact No", "Email")
    
    # Format columns and set widths
    column_widths = {
        'ID': 100,
        'Name': 150,
        'Age': 50,
        'Gender': 100,
        'Country Code': 100,
        'Contact No': 100,
        'Email': 200
    }

    for col in tree["columns"]:
        tree.heading(col, text=col)
        tree.column(col, anchor="center", width=column_widths[col])  # Set width for each column

    # Insert data into treeview
    for row in rows:
        tree.insert("", "end", values=row)

    tree.pack(expand=True, fill='both')

In [5]:
# Function to open 'Manage Courses' window
def manage_courses_window():
    courses_window = tk.Toplevel(root)
    courses_window.title("Manage Courses")

    labels = ['Course Code', 'Course Name', 'Teacher ID']
    entries = {}

    for i, label_text in enumerate(labels):
        label = tk.Label(courses_window, text=label_text)
        label.grid(row=i, column=0, padx=10, pady=5, sticky='e')
        entry = tk.Entry(courses_window)
        entry.grid(row=i, column=1, padx=10, pady=5)
        entries[label_text] = entry

    # Buttons for Insert, View, Delete, and Update
    insert_button = tk.Button(courses_window, text="Insert", command=lambda: insert_course(entries))
    insert_button.grid(row=len(labels), column=0, pady=20)

    view_button = tk.Button(courses_window, text="View", command=view_courses)
    view_button.grid(row=len(labels), column=1, pady=20)

    delete_button = tk.Button(courses_window, text="Delete", command=lambda: delete_course(entries['Course Code'].get()))
    delete_button.grid(row=len(labels)+1, column=0, pady=20)

    update_button = tk.Button(courses_window, text="Update", command=lambda: update_course(entries))
    update_button.grid(row=len(labels)+1, column=1, pady=20)

# Function to insert course records with validation
def insert_course(entries):
    course_code = entries['Course Code'].get().strip()
    
    # Validate course code format (4 letters + 4 digits + 1 letter)
    if not re.match(r'^[A-Z]{4}\d{4}[A-Z]$', course_code):
        messagebox.showerror('Error', 'Course Code must be in the format XXXX0000X!')
        return

    course_name = entries['Course Name'].get().strip()
    teacher_id = entries['Teacher ID'].get().strip()

    # Check if all fields are filled
    if not all([course_code, course_name]):
        messagebox.showerror('Error', 'Please fill in all fields!')
        return

    # Initialize teacher_id as None (for NULL in database)
    teacher_id_to_insert = None

    # Validate if the teacher_id exists in the Teachers table
    conn = sqlite3.connect('education_management.db')
    cursor = conn.cursor()

    cursor.execute("SELECT id FROM Teachers WHERE id = ?", (teacher_id,))
    teacher_exists = cursor.fetchone()

    if teacher_exists:
        teacher_id_to_insert = teacher_id  # Set teacher_id if it exists

    # Insert into database if validation passes
    try:
        cursor.execute('''
            INSERT INTO Courses (course_code, course_name, teacher_id)
            VALUES (?, ?, ?)
            ''', (course_code, course_name, teacher_id_to_insert))

        conn.commit()
        messagebox.showinfo('Success', f'Successfully inserted {course_name} into the database.')

    except sqlite3.IntegrityError:
        messagebox.showerror('Error', 'Course Code must be unique!')
    
    except Exception as e:
        messagebox.showerror('Error', str(e))
    
    finally:
        conn.close()

# Function to delete a course record by Course Code
def delete_course(course_code):
    if not course_code:
        messagebox.showerror('Error', 'Please enter a Course Code to delete!')
        return

    try:
        conn = sqlite3.connect('education_management.db')
        cursor = conn.cursor()

        cursor.execute('DELETE FROM Courses WHERE course_code = ?', (course_code,))
        
        if cursor.rowcount == 0:
            messagebox.showerror('Error', 'Course Code not found!')
        else:
            messagebox.showinfo('Success', f'Successfully deleted course with Course Code {course_code}.')

        conn.commit()
    
    except Exception as e:
        messagebox.showerror('Error', str(e))
    
    finally:
        conn.close()

# Function to update a course record
def update_course(entries):
    course_code = entries['Course Code'].get().strip()
    
    if not course_code:
        messagebox.showerror('Error', 'Please enter a valid Course Code to update!')
        return

    try:
        conn = sqlite3.connect('education_management.db')
        cursor = conn.cursor()

        # Check if the Course Code exists in the database
        cursor.execute("SELECT * FROM Courses WHERE course_code = ?", (course_code,))
        existing_record = cursor.fetchone()
        
        if not existing_record:
            messagebox.showerror('Error', f'Course with Course Code {course_code} does not exist!')
            return

        # Retrieve new values from entries (use original values if blank)
        course_name = entries['Course Name'].get().strip() or existing_record[1]
        teacher_id = entries['Teacher ID'].get().strip()

        # Validate Teacher ID if provided
        teacher_id_to_update = existing_record[2]
        
        if teacher_id:  # If Teacher ID is provided
            cursor.execute("SELECT id FROM Teachers WHERE id = ?", (teacher_id,))
            teacher_exists = cursor.fetchone()
            if not teacher_exists:
                messagebox.showerror('Error', f'Teacher ID {teacher_id} does not exist!')
                return
            teacher_id_to_update = teacher_id  # Update Teacher ID only if valid

        # Update record in the database
        cursor.execute('''
            UPDATE Courses
            SET course_name = ?, teacher_id = ?
            WHERE course_code = ?
            ''', (course_name, teacher_id_to_update, course_code))

        conn.commit()
        messagebox.showinfo('Success', f'Successfully updated course with Course Code {course_code}.')
    
    except Exception as e:
        messagebox.showerror('Error', str(e))
    
    finally:
        conn.close()

# Function to view course records
def view_courses():
    conn = sqlite3.connect('education_management.db')
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM Courses")
    rows = cursor.fetchall()

    # Create a new window to display the records
    view_window = tk.Toplevel(root)
    view_window.title("Courses Table")

    # Create a treeview to display the data
    tree = ttk.Treeview(view_window)
    
    # Define columns
    tree["columns"] = ("Course Code", "Course Name", "Teacher ID")
    
    # Format columns
    for col in tree["columns"]:
        tree.heading(col, text=col)
        tree.column(col, anchor="center")

    # Insert data into treeview
    for row in rows:
        tree.insert("", "end", values=row)

    tree.pack(expand=True, fill='both')

In [6]:
# Function to open 'Manage Enrolled' window
def manage_enrolled_window():
    enrolled_window = tk.Toplevel(root)
    enrolled_window.title("Manage Enrolled")

    labels = ['Course Code', 'Student ID']
    entries = {}

    for i, label_text in enumerate(labels):
        label = tk.Label(enrolled_window, text=label_text)
        label.grid(row=i, column=0, padx=10, pady=5, sticky='e')
        entry = tk.Entry(enrolled_window)
        entry.grid(row=i, column=1, padx=10, pady=5)
        entries[label_text] = entry

    # Buttons for Insert, View and Delete
    insert_button = tk.Button(enrolled_window, text="Insert", command=lambda: insert_enrollment(entries))
    insert_button.grid(row=len(labels), column=0, pady=20)

    view_button = tk.Button(enrolled_window, text="View", command=view_enrollments)
    view_button.grid(row=len(labels), column=1, pady=20)

    delete_button = tk.Button(enrolled_window, text="Delete", command=lambda: delete_enrollment(entries['Course Code'].get(), entries['Student ID'].get()))
    delete_button.grid(row=len(labels)+1, column=0, pady=20)

# Function to insert enrollment records with validation
def insert_enrollment(entries):
    course_code = entries['Course Code'].get().strip()
    student_id = entries['Student ID'].get().strip()

    # Check if all fields are filled
    if not all([course_code, student_id]):
        messagebox.showerror('Error', 'Please fill in all fields!')
        return

    # Validate if the course_code exists in the Courses table
    conn = sqlite3.connect('education_management.db')
    cursor = conn.cursor()

    cursor.execute("SELECT course_code FROM Courses WHERE course_code = ?", (course_code,))
    course_exists = cursor.fetchone()

    if not course_exists:
        messagebox.showerror('Error', 'Course Code does not exist!')
        conn.close()
        return

    # Validate if the student_id exists in the Students table
    cursor.execute("SELECT sid FROM Students WHERE sid = ?", (student_id,))
    student_exists = cursor.fetchone()

    if not student_exists:
        messagebox.showerror('Error', 'Student ID does not exist!')
        conn.close()
        return

    # Insert into database if validation passes
    try:
        cursor.execute('''
            INSERT INTO Enrolled (course_code, sid)
            VALUES (?, ?)
            ''', (course_code, student_id))

        conn.commit()
        messagebox.showinfo('Success', f'Successfully enrolled {student_id} in {course_code}.')

    except sqlite3.IntegrityError:
        messagebox.showerror('Error', 'This enrollment already exists!')
    
    except Exception as e:
        messagebox.showerror('Error', str(e))
    
    finally:
        conn.close()

# Function to delete an enrollment record
def delete_enrollment(course_code, student_id):
    if not course_code or not student_id:
        messagebox.showerror('Error', 'Please enter both Course Code and Student ID to delete!')
        return

    try:
        conn = sqlite3.connect('education_management.db')
        cursor = conn.cursor()

        cursor.execute('DELETE FROM Enrolled WHERE course_code = ? AND sid = ?', (course_code, student_id))
        
        if cursor.rowcount == 0:
            messagebox.showerror('Error', 'Enrollment not found!')
        else:
            messagebox.showinfo('Success', f'Successfully deleted enrollment for {student_id} in {course_code}.')

        conn.commit()
    
    except Exception as e:
        messagebox.showerror('Error', str(e))
    
    finally:
        conn.close()

# Function to view enrollment records
def view_enrollments():
    conn = sqlite3.connect('education_management.db')
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM Enrolled")
    rows = cursor.fetchall()

    # Create a new window to display the records
    view_window = tk.Toplevel(root)
    view_window.title("Enrollments Table")

    # Create a treeview to display the data
    tree = ttk.Treeview(view_window)
    
    # Define columns
    tree["columns"] = ("Course Code", "Student ID")
    
    # Format columns
    for col in tree["columns"]:
        tree.heading(col, text=col)
        tree.column(col, anchor="center")

    # Insert data into treeview
    for row in rows:
        tree.insert("", "end", values=row)

    tree.pack(expand=True, fill='both')


In [7]:
# Main application setup
root = tk.Tk()
root.title("Education Management System")

# Create database tables on startup
create_database()

# Welcome label
welcome_label = tk.Label(root, text="Welcome to the Education Management System", font=("Arial", 16))
welcome_label.pack(pady=20)

# Button to manage students
manage_students_button = tk.Button(root, text="Manage Students", command=manage_students_window)
manage_students_button.pack(pady=10)

# Button to manage teachers
manage_teachers_button = tk.Button(root, text="Manage Teachers", command=manage_teachers_window)
manage_teachers_button.pack(pady=10)

# Button to manage courses
manage_courses_button = tk.Button(root, text="Manage Courses", command=manage_courses_window)
manage_courses_button.pack(pady=10)

# Button to manage enrolled students
manage_enrolled_button = tk.Button(root, text="Manage Enrolled", command=manage_enrolled_window)
manage_enrolled_button.pack(pady=10)

# Run the application
root.mainloop()